In [ ]:
import prep
import pickle
from easydict import EasyDict as edict
from parser.helper.spanoverlap_data_module import DataModuleForSpanInfoGain, DataModuleForGPTSamples
from ipynb_spanoverlap.utils import SpacyLemmatizer 

In [ ]:
import sys
from models import Mistral7BEnhancer, SpanScorer, GPTEnhancer

In [ ]:

import scipy
import random
import torch
import numpy as np
from utils import compute_sentence_logprob, cyk

In [ ]:

with open("/fast/mdxuser/projects/tn-pcfg/data/ctb5/ctb-dev/test.gpt35.full-reorder_q_confirm_q_wh_tense-zh_prompt-0125.pickle", 'rb') as f:
    data = pickle.load(f)

In [ ]:
gpt_model = GPTEnhancer()

In [ ]:
class SpanScorer:
    def __init__(self) -> None:
        # self.form = form
        pass

    def score_by_count(
        self,
        form,
        samples,
        stemmer,
        flag_rm_spaces_in_matching=False,
        flag_print_samples=False,
    ):
        from ipynb_spanoverlap.utils import normalizing_string
        assert isinstance(form, list), f"{form} must be pre-tokenized"

        # print("form:", form)
        # for s in samples:
        # print(s)
        # print("----sep----")
        # print("score count:", stemmer)
        cleaned_samples = [normalizing_string(i, stemmer) for i in samples]
        if flag_print_samples:
            print(form)
            for s in cleaned_samples:
                print(s)

        if flag_rm_spaces_in_matching:
            cleaned_samples = [i.replace(" ", "") for i in samples]
        scores = np.zeros((len(form) + 1, len(form) + 1))
        for i in range(len(form)):
            for j in range(i + 2, len(form) + 1):
                span_literal = " ".join(form[i:j])
                if flag_rm_spaces_in_matching:
                    span_literal = span_literal.replace(" ", "")
                scores[i, j] = sum([span_literal in samp for samp in cleaned_samples])
        return scores

    def score_by_meteor(
        self,
        form,
        samples,
        stemmer,
        flag_rm_spaces_in_matching=False,
        flag_print_samples=False,
    ):
        from ipynb_spanoverlap.utils import normalizing_string
        from nltk import word_tokenize
        import nltk
        assert isinstance(form, list), f"{form} must be pre-tokenized"

        # print("form:", form)
        # for s in samples:
        # print(s)
        # print("----sep----")
        # print("score count:", stemmer)
        meteor_scorer = lambda substr, span: nltk.translate.meteor_score.single_meteor_score(substr, span, alpha = 0.02, gamma=0.9, beta = 5)
        cleaned_samples = [normalizing_string(i, stemmer) for i in samples]
        if flag_print_samples:
            print(form)
            for s in cleaned_samples:
                print(s)

        if flag_rm_spaces_in_matching:
            cleaned_samples = [i.replace(" ", "") for i in samples]
        scores = np.zeros((len(form) + 1, len(form) + 1))
        for i in range(len(form)):
            for j in range(i + 2, len(form) + 1):
                span_literal = " ".join(form[i:j])
                if flag_rm_spaces_in_matching:
                    span_literal = span_literal.replace(" ", "")
                sample_scores = []
                for samp in cleaned_samples:
                    tokenized_samp = word_tokenize(samp)
                    matching_scores = [meteor_scorer(tokenized_samp[p: p+j-i], form[i:j]) for p in range(len(samp)-j+i)]
                    sample_scores.append(max(matching_scores) if len(matching_scores) > 0 else 0)
                scores[i, j] = sum(sample_scores)
        return scores

    def score_by_bleu(
        self,
        form,
        samples,
        stemmer,
        flag_rm_spaces_in_matching=False,
        flag_print_samples=False,
    ):
        from ipynb_spanoverlap.utils import normalizing_string
        from nltk import word_tokenize
        import nltk
        assert isinstance(form, list), f"{form} must be pre-tokenized"

        # print("form:", form)
        # for s in samples:
        # print(s)
        # print("----sep----")
        # print("score count:", stemmer)
        chencherry = nltk.translate.bleu_score.SmoothingFunction()
        meteor_scorer = lambda substr, span: nltk.translate.bleu_score.sentence_bleu(substr, span, smoothing_function=chencherry.method3)
        cleaned_samples = [normalizing_string(i, stemmer) for i in samples]
        if flag_print_samples:
            print(form)
            for s in cleaned_samples:
                print(s)

        if flag_rm_spaces_in_matching:
            cleaned_samples = [i.replace(" ", "") for i in samples]
        scores = np.zeros((len(form) + 1, len(form) + 1))
        for i in range(len(form)):
            for j in range(i + 2, len(form) + 1):
                span_literal = " ".join(form[i:j])
                if flag_rm_spaces_in_matching:
                    span_literal = span_literal.replace(" ", "")
                sample_scores = []
                for samp in cleaned_samples:
                    tokenized_samp = word_tokenize(samp)
                    matching_scores = [meteor_scorer([tokenized_samp[p: p+j-i]], form[i:j]) for p in range(len(samp)-j+i)]
                    sample_scores.append(max(matching_scores) if len(matching_scores)>0 else 0)
                scores[i, j] = sum(sample_scores)
        return scores





In [ ]:
span_scorer = SpanScorer()

In [ ]:
data['prompts'][10]

In [ ]:

commonpath = "/fast/mdxuser/projects/tn-pcfg/data/SPMRL_SHARED_2014_NO_ARABIC"
hparams = edict({
    'device': 'cuda:0',
    'data':{
        # 'val_file':'/fast/mdxuser/projects/tn-pcfg/data/test.gpt35.full-reorder_q_confirm_q_wh_clefting_perfect-1106.pickle' #ptb23
        # 'val_file':'/fast/mdxuser/projects/tn-pcfg/data/ptb/ptb-val/test.gpt35.full-reorder_q_confirm_q_wh_clefting_tense-0125.pickle' #ptb22
        # 'val_file':'/fast/mdxuser/projects/tn-pcfg/data/ptb/ptb-val/test.gpt35.full-reorder_q_confirm_q_wh_clefting_perfect-gpt4-0125.pickle' #ptb22
        # 'val_file':'/fast/mdxuser/projects/tn-pcfg/data/ptb/ptb-test/test.gpt35.full-reorder_q_confirm_q_wh_clefting_tense-0125.pickle' #ptb22
        # 'val_file':'/fast/mdxuser/projects/tn-pcfg/data/ctb5/ctb-test/test.gpt35.full-reorder_q_confirm_q_wh_tense-zh_prompt-0125.pickle' #ctb7 test
        # 'val_file':'/fast/mdxuser/projects/tn-pcfg/data/ctb5/ctb-dev/test.gpt35.full-reorder_q_confirm_q_wh_tense-zh_prompt-0125.pickle' #ctb7 test
        # 'val_file':f'{commonpath}/GERMAN_SPMRL/gold/ptb/test/test.gpt35.full-reorder_q_confirm_q_wh_tense-de_prompt-0125.pickle'
        'val_file':f'{commonpath}/GERMAN_SPMRL/gold/ptb/dev/test.gpt35.full-reorder_q_confirm_q_wh_tense-de_prompt-0125.pickle'
        # 'val_file':f'{commonpath}/KOREAN_SPMRL/gold/ptb/test/test.gpt35.full-reorder_q_confirm_q_wh_clefting_perfect-0125.pickle'
        # 'val_file':f'{commonpath}/KOREAN_SPMRL/gold/ptb/test/test.gpt35.full-reorder_q_confirm_q_wh_tense-kr_prompt-0125.pickle'
        # 'val_file':f'{commonpath}/KOREAN_SPMRL/gold/ptb/dev/test.gpt35.full-reorder_q_confirm_q_wh_tense-kr_prompt-0125.pickle'
        # 'val_file':'{commonpath}/FRENCH_SPMRL/gold/ptb/test/test.gpt35.full-reorder-0125.pickle'
        # 'val_file':f'{commonpath}/FRENCH_SPMRL/gold/ptb/test/test.gpt35.full-reorder_q_confirm_q_wh_tense-fr_prompt-0125.pickle'
        # 'val_file':f'{commonpath}/FRENCH_SPMRL/gold/ptb/dev/test.gpt35.full-reorder_q_confirm_q_wh_tense-fr_prompt-0125.pickle'
        # 'val_file':f'{commonpath}/HUNGARIAN_SPMRL/gold/ptb/dev/test.gpt35.full-reorder_q_confirm_q_wh_tense-hu_prompt-0125.pickle'
        # 'val_file':f'{commonpath}/HUNGARIAN_SPMRL/gold/ptb/test/test.gpt35.full-reorder_q_confirm_q_wh_tense-hu_prompt-0125.pickle'
        # 'val_file':f'{commonpath}/POLISH_SPMRL/gold/ptb/test/test.gpt35.full-reorder_q_confirm_q_wh_tense-pl_prompt-0125.pickle'
        # 'val_file':f'{commonpath}/POLISH_SPMRL/gold/ptb/dev/test.gpt35.full-reorder_q_confirm_q_wh_tense-pl_prompt-0125.pickle'
        # 'val_file':f'{commonpath}/SWEDISH_SPMRL/gold/ptb/test/test.gpt35.full-reorder_q_confirm_q_wh_tense-sv_prompt-0125.pickle'
        # 'val_file':f'{commonpath}/SWEDISH_SPMRL/gold/ptb/dev/test.gpt35.full-reorder_q_confirm_q_wh_tense-sv_prompt-0125.pickle'
        # 'val_file':f'{commonpath}/HEBREW_SPMRL/gold/ptb/test/test.gpt35.full-reorder_q_confirm_q_wh_tense-he_prompt-0125.pickle'
        # 'val_file':f'{commonpath}/HEBREW_SPMRL/gold/ptb/dev/test.gpt35.full-reorder_q_confirm_q_wh_tense-he_prompt-0125.pickle'
        # 'val_file':f'{commonpath}/BASQUE_SPMRL/gold/ptb/test/test.gpt35.full-reorder_q_confirm_q_wh_tense-baq_prompt-gpt4.pickle'
        # 'val_file':f'{commonpath}/BASQUE_SPMRL/gold/ptb/dev/test.gpt35.full-reorder_q_confirm_q_wh_tense-baq_prompt-gpt4.pickle'
        # 'val_file':f'{commonpath}/BASQUE_SPMRL/gold/ptb/test/test.gpt35.full-reorder_q_confirm_q_wh_clefting_perfect-gpt4_0125.pickle'
    },
    'test':{
        'batch_size':4
    }
})
lang_str = "german"

In [ ]:
class PolishStemmer():
    def __init__(self) -> None:
        from stempel import StempelStemmer
        self.stemmer = StempelStemmer.polimorf()
        pass
    
    def stemWord(self, word):
        return self.stemmer.stem(word)

class identityStemmer():
    def __init__(self) -> None:
        pass
    
    def stemWord(self, word):
        return word

In [ ]:
# from pystempel import Stemmer as PolishStemmer

In [ ]:
from nltk.stem.snowball import *
# from utils import HebrewLemmatizer
from Stemmer import Stemmer #snowball stemmer
if lang_str == "polish":
    stemmer = PolishStemmer()
elif lang_str in ['hebrew', 'chinese', 'korean']:
    stemmer = identityStemmer()
# elif lang_str == "korean":
    # stemmer = identityStemmer() 

# elif lang_str == "korean":
    # stemmer = SpacyLemmatizer(lang_str) 
else:
    stemmer = Stemmer(lang_str)

In [ ]:

dst = DataModuleForGPTSamples(hparams=hparams)
it = dst.dev_dataloader(max_len=100, min_len=1, langstr=lang_str, prompt_type = ['reorder_diverse'])

In [ ]:
from nltk import PorterStemmer
ps = PorterStemmer()

In [ ]:
from utils import normalizing_string
def normalizing_permuted_en(x):
    return x.replace("n't", " n't").replace("'s", " 's").replace('%', ' %').replace('$', '$ ').replace('(', ' -LRB- ').replace(')', ' -RRB- ').replace("'m", " 'm").replace("n't", "not")

def normalizing_permuted_fr(x):
    return x.replace("l'", "l' ").replace("d'", "d' ").replace('n\'', 'n\' ').replace('(', ' -LRB- ').replace(')', ' -RRB- ')#.replace('l\'', 'les').replace('d\'', 'das')

def normalizing_permuted(x, lang_str): 
    if lang_str == "french":
        # return x
        return normalizing_permuted_fr(x)
    elif lang_str == "english":
        return normalizing_permuted_en(x)
    else:
        return x

In [ ]:
def accumulate_const_stats_excluderb(accu_recall, accu_length, accu_count, labeled_spans, scores, overlap):
    rb_tree = [(i, len(scores)-1) for i in range(1, len(scores)-2)]
    for s in labeled_spans:
        label, span = s[2], s[:2]
        label = label.split('-')[0]
        label = label.split('##')[0]

        if lang_str == 'french':
            if label in ['VPpart', 'VPinf']: label= 'VP'
            if label in ['Srel', 'Sint', 'Ssub']: label = 'S'
        elif lang_str == 'polish':
            if label == 'zdanie': label ='S'
            if label == 'fwe': label = 'VP'
            if label == 'fno': label = 'NP'
        elif lang_str == 'basque':
            if label == 'SN': label = 'NP'
            if label == 'SP': label = 'PP'
            # if
            # 'zdanie', 'fwe', 'fno'


        if span[1] - span[0] == len(scores)-1: continue
        if span[1] - span[0] == 1: continue

        if label not in accu_count.keys():
            accu_count[label] = []
            accu_length[label] = []
            accu_recall[label] = np.array((0, 0, 0))

        if span in overlap and span not in rb_tree:
            accu_recall[label]+=np.array((1, 1, 0))
        elif span in overlap and span in rb_tree:
            accu_recall[label]+=np.array((0, 1, 1))
        else:
            accu_recall[label]+=np.array((0, 1, 0))
        
        accu_count[label].append(scores[*span])
        assert not np.isnan(scores[*span]), "score should not be nan"
        accu_length[label].append(span[1]-span[0])


def accumulate_scores_for_spans(accu, spans, score): 
    for s in spans:
        accu[s[1]-s[0]].append(score[*s])
    # accu.extend([score[*s] for s in spans])

def accumulate_margins(accu, gs, rs, score):
    for s in gs:
        if s[1]-s[0] == len(score)-1:continue
        len_span = s[1]-s[0]
        competing_spans = [(i, i+len_span) for i in range(max(0, s[0]-len_span+1), min(s[1], len(score)-len_span)) if i != s[0]]
        candidate_scores = [score[*sprime] for sprime in competing_spans]

        # candidate_set = [score[*sprime] for sprime in rs if s[1]-s[0] == sprime[1]-sprime[0]]
        # print(competing_spans)
        # print(len_span, len(score)-1)
        sprime = max(candidate_scores)
        accu[s[1]-s[0]].append(score[*s] - sprime)


In [ ]:
list_remove_spaces = ["chinese", "hebrew"]

In [ ]:
suffix = 'he'

In [ ]:
flag_remove_spaces_in_matching = lang_str in list_remove_spaces 
flag_debug = False
flag_random_score = False 
flag_rb_score = False 
# prompt_type = ['reorder_diverse']
# prompt_type = ['confirmatory_question_diverse', 'wh_question_diverse']
# prompt_type = ['perfect_diverse']
# prompt_type = ['clefting_diverse']
# prompt_type = ['reorder_diverse', 'perfect_diverse', 'clefting_diverse']
# prompt_type = ['reorder_diverse_baq', 'tense_diverse_baq']
# prompt_type = [f'reorder_diverse{suffix}']
# prompt_type = [f'confirmatory_question_diverse{suffix}', f'wh_question_diverse{suffix}']
# prompt_type = [f'tense_diverse{suffix}']
prompt_type = 'anything'
# print(dst.val_dataset.payload.keys())
# dst.val_dataset.payload = {k: v[:100] for k, v in dst.val_dataset.payload.items()}

In [ ]:
from tqdm import tqdm
from copy import deepcopy
it = dst.dev_dataloader(max_len=10, min_len=1, langstr=lang_str, prompt_type=prompt_type)
macro_f1_info = []
macro_f1_count = []
gold_scores = [[] for i in range(100)]
random_scores = [[] for i in range(100)]
const_label_count = {}
const_label_recall = {}
const_label_length = {}
corpus_f1_counter = np.zeros(3)
for x, y in tqdm(it):
    # print(x.keys(), y.keys())
    batch_size = len(x['form_array'])
    for bid in range(batch_size):
        if flag_debug: print('============')
        form = x['form_array'][bid]
        pos = x['pos_array'][bid]
        labeled_gold_tree = y['gold_tree'][bid]
        non_trivial_labeled_gold_tree = set([tuple(s[:2]) for s in labeled_gold_tree if s[1]-s[0] < len(form) and s[1]-s[0]>1])

        gpt_samples = x['gpt_sample_array'][bid]
        # print(gpt_samples)
        gpt_samples = [normalizing_permuted(s, lang_str) for s in gpt_samples]
        if len(form) == 1: 
            # print(non_trivial_labeled_gold_tree)
            macro_f1_count.append(1)
        if len(gpt_samples) == 0: 
            # print("no samples")
            # print(len(non_trivial_labeled_gold_tree))
            macro_f1_count.append(0)
            continue


        scores_count = span_scorer.score_by_meteor([normalizing_string(w, stemmer) for w in form], gpt_samples, stemmer, 
                                                  flag_rm_spaces_in_matching=flag_remove_spaces_in_matching,
                                                  flag_print_samples=flag_debug)
        if flag_random_score:
            scores_count = np.random.uniform(size = scores_count.shape)
        if flag_rb_score:
            scores_count = np.ones(scores_count.shape)
        _, _, spans_count = cyk(scores_count, rb_bias=True)
        spans_count = [s for s in spans_count if s[1]-s[0]<len(form)]
        
        non_trivial_random_tree  = set([tuple(s[:2]) for s in y['random_tree'][bid] if s[1]-s[0] < len(form) and s[1]-s[0]>1])
        
        def compute_f1(pred, gold):
            spans = pred
            non_trivial_labeled_gold_tree = gold
            overlap = set(spans).intersection(set(non_trivial_labeled_gold_tree))
            prec = float(len(overlap)) / (len(set(non_trivial_labeled_gold_tree)) + 1e-8)
            reca = float(len(overlap)) / (len(set(spans)) + 1e-8)
            if len(set(non_trivial_labeled_gold_tree)) == 0:
                reca = 1.
                if len(set(spans)) == 0:
                    prec = 1.
            f1 = 2 * prec * reca / (prec + reca + 1e-8)
            return f1

        def compute_corpus_f1_counter(pred, gold):
            overlap = set(pred).intersection(set(gold))
            tp = len(overlap)
            fp = len(pred) - len(overlap)
            fn = len(gold) - len(overlap)
            return np.array((tp, fp, fn))

        f1_count = compute_f1(spans_count, non_trivial_labeled_gold_tree)
        corpus_f1_counter += compute_corpus_f1_counter(spans_count, non_trivial_labeled_gold_tree)
        macro_f1_count.append(f1_count)
        accumulate_const_stats_excluderb(const_label_recall, const_label_length, const_label_count,
                                         labeled_gold_tree, scores_count/len(gpt_samples), set(spans_count).intersection(non_trivial_labeled_gold_tree))
        accumulate_margins(gold_scores, non_trivial_labeled_gold_tree, non_trivial_random_tree, scores_count)
        # print(non_trivial_labeled_gold_tree)
        # print(non_trivial_random_tree)
        # accumulate_scores_for_spans(random_scores, non_trivial_random_tree, scores_count)

        if False or (flag_debug and f1_count < 0.4):
            print("----------")
            # print(form)
            # print(" ".join(form))
            form = [normalizing_string(w, stemmer) for w in form]
            print(form)
            print(pos)
            # print(gpt_samples)
            print(f1_count)
            print('---------count---------')
            for s in spans_count:
                print(form[slice(*s)],  scores_count[*s], s in non_trivial_labeled_gold_tree)

            print("--------gold---------")

            for s in non_trivial_labeled_gold_tree:
                # print(s[2])
                # print(s)
                # s = s[:2]
                print(" ".join(form[s[0]: s[1]]), scores_count[s[0], s[1]])

 
print(np.mean(macro_f1_count))

        

In [ ]:
def compute_f1_from_counter(f1_counter):
    tp, fp, fn = f1_counter
    prec = tp / (tp + fp + 1e-8)
    reca = tp / (tp + fn + 1e-8)
    f1 = 2 * prec * reca / (prec + reca + 1e-8)
    return f1
compute_f1_from_counter(corpus_f1_counter)
# print(corpus_f1_counter[0]+corpus_f1_counter[1])

In [ ]:
gold_mean = [np.mean(gold_scores[i]) for i in range(2, 30) if len(gold_scores[i]) > 200]
gold_std = [np.std(gold_scores[i]) for i in range(2, 30)  if len(gold_scores[i]) > 200]
proportion = [sum(np.array(gold_scores[i])>0)/len(gold_scores[i]) for i in range(2, 30)  if len(gold_scores[i]) > 100]
# print([np.mean(gold_scores[i]) for i in range(2, 30)])

In [ ]:
# for len_2, (mu, sigma) in enumerate(zip(gold_mean, gold_std)):
    # print(len_2+2, 1-scipy.stats.norm.cdf(-mu/sigma))


for len_2, prop in enumerate(proportion):
    print(len_2+2, prop)


# [ ()]

In [ ]:
# [ (len_2+2, scipy.stats.norm.cdf(-(-mu)/sigma)) for len_2, (mu, sigma) in enumerate(zip(gold_mean, gold_std))]

In [ ]:
# suffix = '_kr'
# from tqdm import tqdm
# from copy import deepcopy
# # for prompt_type in [ ['clefting_diverse'],]:
# for prompt_type in [[f'confirmatory_question_diverse{suffix}', f'wh_question_diverse{suffix}'], [f'reorder_diverse{suffix}'], [f'tense_diverse{suffix}'], 'anything']:
#     macro_f1_info = []
#     macro_f1_count = []
#     const_label_count = {}
#     const_label_recall = {}
#     const_label_length = {}
#     it = dst.dev_dataloader(max_len=100, min_len=1, langstr=lang_str, prompt_type=prompt_type)
#     corpus_f1_counter = np.zeros(3)
#     for x, y in tqdm(it):
#         # print(x.keys(), y.keys())
#         batch_size = len(x['form_array'])
#         for bid in range(batch_size):
#             if flag_debug: print('============')
#             form = x['form_array'][bid]
#             pos = x['pos_array'][bid]
#             labeled_gold_tree = y['gold_tree'][bid]

#             gpt_samples = x['gpt_sample_array'][bid]
#             gpt_samples = [normalizing_permuted(s, lang_str) for s in gpt_samples]
#             if len(form) == 1: macro_f1_count.append(1)
#             if len(gpt_samples) == 0: 
#                 macro_f1_count.append(0)
#                 continue


#             scores_count = span_scorer.score_by_count([normalizing_string(w, stemmer) for w in form], gpt_samples, stemmer, 
#                                                     flag_rm_spaces_in_matching=flag_remove_spaces_in_matching,
#                                                     flag_print_samples=flag_debug)
#             _, _, spans_count = cyk(scores_count, rb_bias=True)
#             spans_count = [s for s in spans_count if s[1]-s[0]<len(form)]
            
#             non_trivial_labeled_gold_tree = set([tuple(s[:2]) for s in labeled_gold_tree if s[1]-s[0] < len(form) and s[1]-s[0]>1])
            
#             def compute_f1(pred, gold):
#                 spans = pred
#                 non_trivial_labeled_gold_tree = gold
#                 overlap = set(spans).intersection(set(non_trivial_labeled_gold_tree))
#                 prec = float(len(overlap)) / (len(set(non_trivial_labeled_gold_tree)) + 1e-8)
#                 reca = float(len(overlap)) / (len(set(spans)) + 1e-8)
#                 if len(set(non_trivial_labeled_gold_tree)) == 0:
#                     reca = 1.
#                     if len(set(spans)) == 0:
#                         prec = 1.
#                 f1 = 2 * prec * reca / (prec + reca + 1e-8)
#                 return f1

#             def compute_corpus_f1_counter(pred, gold):
#                 overlap = set(pred).intersection(set(gold))
#                 tp = len(overlap)
#                 fp = len(pred) - len(overlap)
#                 fn = len(gold) - len(overlap)
#                 return np.array((tp, fp, fn))

#             f1_count = compute_f1(spans_count, non_trivial_labeled_gold_tree)
#             corpus_f1_counter += compute_corpus_f1_counter(spans_count, non_trivial_labeled_gold_tree)
#             macro_f1_count.append(f1_count)
#             accumulate_const_stats_excluderb(const_label_recall, const_label_length, const_label_count,
#                                             labeled_gold_tree, scores_count/len(gpt_samples), set(spans_count).intersection(non_trivial_labeled_gold_tree))


#             if False or (flag_debug and f1_count < 0.4):
#                 print("----------")
#                 # print(form)
#                 # print(" ".join(form))
#                 form = [normalizing_string(w, stemmer) for w in form]
#                 print(form)
#                 print(pos)
#                 # print(gpt_samples)
#                 print(f1_count)
#                 print('---------count---------')
#                 for s in spans_count:
#                     print(form[slice(*s)],  scores_count[*s], s in non_trivial_labeled_gold_tree)

#                 print("--------gold---------")

#                 for s in non_trivial_labeled_gold_tree:
#                     # print(s[2])
#                     # print(s)
#                     # s = s[:2]
#                     print(" ".join(form[s[0]: s[1]]), scores_count[s[0], s[1]])

    
#     print(prompt_type, np.mean(macro_f1_count))

            

In [ ]:
const_label_recall.keys()

In [ ]:
import pandas as pd

In [ ]:
label_list = {
    "english": ["S", "VP", "NP", "PP", "QP"],
    "chinese": ["VP", "IP", "CP", "NP", "PP", "QP"],
    'german': ['S', 'VP', 'NP', 'PP', 'AP'],
    'french': ['S', 'VP',  'NP', 'PP', 'AP' ],
    'hungarian': ['CP', 'NP', 'PP'],
    'swedish': ['S', 'VP', 'NP', 'PP', 'AP'],
    'polish': ['S', 'VP', 'NP'],
    'basque': ['S',  'NP', 'PP'],
    'hebrew': ['S', 'VP', 'NP', 'PP'],
    'korean': ['VP', 'NP']
}

In [ ]:
import scipy
import matplotlib.pyplot as plt
recall_count_pair = []
mean_len_pair = {k:[] for k in label_list[lang_str]}
skewness_len_pair = {k:[] for k in label_list[lang_str]}
# df_mean_len = 

print('label, delta-recall, mean-span-overlap')
for l_target in range(2, 15):
    print('length:', l_target)
    for k, v in const_label_recall.items():
        # print(k)
        if v[1]<=20: continue
        if lang_str == 'french':

            if k not in label_list[lang_str]: continue
        else:
            if k not in label_list[lang_str]: continue
        # print(k, v[0]/v[1], (v[0]+v[2])/v[1], np.mean(const_label_count[k]))
        # print(const_label_count[k][:10])
        datapoints = [c for l, c in zip(const_label_length[k], const_label_count[k]) if l == l_target]
        print(k, len(datapoints))
        if len(datapoints) < 20: continue
        print(k, np.mean(datapoints), scipy.stats.skew(datapoints), len(datapoints))
        mean_len_pair[k].append(np.mean(datapoints))
        skewness_len_pair[k].append(scipy.stats.skew(datapoints))
        # recall_count_pair.append(((v[0]+v[2])/v[1], np.mean(const_label_count[k]), k))
    print('====')

In [ ]:
from scipy.interpolate import make_interp_spline, BSpline
from matplotlib.ticker import MaxNLocator

max_len = max([len(v) for v in mean_len_pair.values()])
# xnew = np.linspace(0, max_len, num=300, endpoint=True)
# T = range(max_len)
# spl = make_interp_spline(T, power, k=3)  # type: BSpline
# power_smooth = spl(xnew)


# for ax in axs:
# plt.axes().xaxis.set_major_locator(MaxNLocator(integer=True)

fig, axs = plt.subplots(1, 2, figsize = (8, 3), dpi=140)
# plt.axes().xaxis.set_major_locator(MaxNLocator(integer=True))
for ax in axs:
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))

df_mean_len = pd.DataFrame.from_dict({k: np.pad(np.array(v), (0, max_len-len(v)),mode='constant',constant_values=(np.nan,)) for k, v in mean_len_pair.items()})
# df_mean_len.set_index([pd.Index(range(2, 15))])
df_mean_len.index +=2
print(df_mean_len)
df_mean_len.plot(ax = axs[0])
axs[0].set_xlabel('Span Length')
axs[0].set_ylabel('Mean Span-overlap Score')
df_skewness_len = pd.DataFrame.from_dict({k: np.pad(np.array(v), (0, max_len-len(v)),mode='constant',constant_values=(np.nan,)) for k, v in skewness_len_pair.items()})
df_skewness_len.index +=2
print(df_skewness_len)
df_skewness_len.plot(ax = axs[1])
axs[1].set_xlabel('Span Length')
axs[1].set_ylabel('Skewness Value')
plt.tight_layout()

In [ ]:
plt.bar([i for i in range(len(recall_count_pair))], [i[1] for i in recall_count_pair], width=0.8)
ax = plt.gca()
ax.set_xticks([i for i in range(len(recall_count_pair))])
ax.set_xticklabels([i[2] for i in recall_count_pair], rotation = 45)

In [ ]:


from matplotlib import pyplot as plt
# X=np.random.choice(np.array(const_label_count['QP']), 128, replace=False)
for k in sorted(const_label_count.keys()):
    # if k == "S": continue
    if k not in label_list[lang_str]: continue
    X = const_label_count[k]
    if len(X)<100: continue
    
    counts, bins = np.histogram(X, bins=25)
    fig, axs = plt.subplots(1, figsize=(5, 3))
    # print
    if not isinstance(axs, list): axs = [axs]
    axs[0].stairs(counts/np.sum(counts), bins)
    axs[0].set_title(k)
    axs[0].set_xlabel('span-overlap scores')
    axs[0].set_ylabel('constituent proportion')

    plt.show()

In [ ]:

fig, axs = plt.subplots(2, 3, figsize=(10, 6), dpi=140)
# plt.bar([i for i in range(len(recall_count_pair))], [i[1] for i in recall_count_pair], width=0.8)
filtered_recall_count_pair = [i for i in recall_count_pair if i[2] in label_list[lang_str]]
axs[0, 0].bar([i for i in range(len(filtered_recall_count_pair))], [i[1] for i in filtered_recall_count_pair], width=0.8)
# ax = plt.gca()
axs[0, 0].set_xticks([i for i in range(len(filtered_recall_count_pair))])
axs[0, 0].set_xticklabels([i[2] for i in filtered_recall_count_pair], rotation = 45)
axs[0, 0].set_xlabel("Constituent Types")
axs[0, 0].set_ylabel("Mean Span-overlap Scores")
axs[0, 0].set_title("Mean Span-overlap Scores")
 
cnt=1
for k in label_list[lang_str]:
    # if k == "S": continue
    if k not in label_list[lang_str]: continue
    X = const_label_count[k]
    if len(X)<100: continue

    ax_idx = [int(cnt/3), int(cnt%3)]
    print(ax_idx)
    
    counts, bins = np.histogram(X, bins=25)
    # fig, axs = plt.subplots(1, figsize=(5, 3))
    # print
    # if not isinstance(axs, list): axs = [axs]
    axs[ax_idx[0], ax_idx[1]].stairs(counts/np.sum(counts), bins)
    axs[ax_idx[0], ax_idx[1]].set_title(k)
    axs[ax_idx[0], ax_idx[1]].set_xlabel('span-overlap scores')
    axs[ax_idx[0], ax_idx[1]].set_ylabel('constituent proportion')
    axs[ax_idx[0], ax_idx[1]].text(.05 if int(cnt/3)>0 else 0.4, .9, "Mean Score = {:.2f}".format(np.mean(X)), transform=axs[ax_idx[0], ax_idx[1]].transAxes)
    axs[ax_idx[0], ax_idx[1]].text(.05if int(cnt/3)>0 else 0.4, .8, "Skewness = {:.2f}".format(scipy.stats.skew(X)), transform=axs[ax_idx[0], ax_idx[1]].transAxes)
    cnt+=1

    # plt.show()
plt.tight_layout()
